In [30]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import re
from shutil import copy2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pocketsphinx

### Bag 0 : Trial n Error

In [23]:
def load_images_from_folder(folder):
    images = []
    filenames = []
    
    def extract_id(filename):
        match = re.search(r'id_(\d+)_', filename)
        return int(match.group(1)) if match else -1
    
    file_list = [f for f in os.listdir(folder) if f.endswith(".jpg") or f.endswith(".png")]
    file_list.sort(key=extract_id)
    
    for filename in file_list:
        img_path = os.path.join(folder, filename)
        img = Image.open(img_path).convert('RGB') 
        img = img.resize((128, 128))
        img_array = np.array(img)
        images.append((img_array, filename))
    
    return images

folder_path = r"C:\Users\gagah\Desktop\KKN\data\Train Images 13440x32x32\train"
images_with_filenames = load_images_from_folder(folder_path)

images = np.array([img[0] for img in images_with_filenames])
filenames = [img[1] for img in images_with_filenames]

print(f'Total data gambar: {len(images)}')
print(f'Shape setiap gambar: {images[0].shape}')

Total data gambar: 13440
Shape setiap gambar: (128, 128, 3)


In [ ]:
# Memisahkan gambar berdasarkan label
label_dict = {}
for img_array, filename in images_with_filenames:
    match = re.search(r'label_(\d+)', filename)
    if match:
        label = int(match.group(1))
        if label not in label_dict:
            label_dict[label] = []
        label_dict[label].append((img_array, filename))

# Menampilkan gambar per label
for label, images in label_dict.items():
    fig, axes = plt.subplots(1, 8, figsize=(10, 2))
    axes = axes.ravel()
    for i in range(min(8, len(images))):  # Hanya menampilkan maksimal 6 gambar per label
        axes[i].imshow(images[i][0])
        axes[i].axis('off')
        axes[i].set_title(images[i][1], fontsize=8)
    plt.suptitle(f'Label {label}')
    plt.tight_layout()
    plt.show()

In [38]:
def create_folders_by_label(source_folder, destination_folder):
    # Membuat folder tujuan jika belum ada
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    file_list = [f for f in os.listdir(source_folder) if f.endswith(".jpg") or f.endswith(".png")]
    
    for filename in file_list:
        match = re.search(r'id_(\d+)_label_(\d+)', filename)
        if match:
            img_id = match.group(1)
            label = match.group(2)
            
            # Membuat path folder label
            label_folder = os.path.join(destination_folder, f'lab_{label}')
            if not os.path.exists(label_folder):
                os.makedirs(label_folder)
            
            # Menyalin file ke folder tujuan dengan nama file baru
            src_path = os.path.join(source_folder, filename)
            dest_path = os.path.join(label_folder, f'id_{img_id}.png')
            copy2(src_path, dest_path)

source_folder_path = r"C:\Users\gagah\Desktop\KKN\data\Train Images 13440x32x32\train"
destination_folder_path = r"C:\Users\gagah\Desktop\KKN\data\Data_Train_Images"

create_folders_by_label(source_folder_path, destination_folder_path)


### Bag 1 : Classification Hijaiyah

In [ ]:
    TRAINING_DIR = "data/rps"
    training_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=2,
        shear_range=0.4
    )

    validation_datagen = ImageDataGenerator(
        rescale=1./255
    )

    # YOUR IMAGE SIZE SHOULD BE 150x150
    # Make sure you used "categorical"
    train_generator = training_datagen.flow_from_directory(
        directory=TRAINING_DIR,
        target_size=(150, 150),
        class_mode='categorical'
    )

    validation_generator = validation_datagen.flow_from_directory(
        directory=TRAINING_DIR,
        target_size=(150, 150),
        class_mode='categorical'
    )

    model = tf.keras.models.Sequential([
    # YOUR CODE HERE, end with 3 Neuron Dense, activated by softmax
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    callback = myCallback()

    model.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator,
        callbacks=[callback]
    )

In [6]:



# Load and preprocess dataset
# ... (data loading and preprocessing steps)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(28, activation='softmax')  # Assuming 28 classes for Hijaiyah letters
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# Train the model
# train_images, train_labels should be preprocessed datasets
model.fit(datagen.flow(train_images, train_labels, batch_size=32), epochs=10, validation_data=(val_images, val_labels))

# Save the model
model.save('hijaiyah_model.h5')


### Bag 2 : Recognize Doa Speech